# Лекция 1: Вступление

## Полезные ссылки на повторение матана

1. [Linear Algebra](http://cs229.stanford.edu/section/cs229-linalg.pdf)
2. [Probability Theory](http://cs229.stanford.edu/section/cs229-prob.pdf)
3. [SGD](http://cs231n.github.io/optimization-1/)
4. [Convex Optimization](http://cs229.stanford.edu/section/cs229-cvxopt.pdf)

## Конспект

[Здесь](http://web.stanford.edu/class/cs224n/lecture_notes/cs224n-2017-notes1.pdf)

# Лекция 2: Word2Vec

## Полезно почитать

* [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
* [Mikolov "Distributed Representations of Words and Phrases and their Compositionality"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
* [Mikolov "Efficient Estimation of Word Representations in Vector Space"](https://arxiv.org/pdf/1301.3781.pdf)

## W2V

__Задача:__ $$ p(context \: | \: w_t) $$

__Функция стоимости:__ $$ J = 1 - p(w_{-t} \:|\: w_t) $$, где $_{-t}$ значит _"все слова контекста, кроме $t$"_.

Два алгоритма:

1. __Skip-grams (SG)__ - _будет разобран далее_.
   
   Предсказывать контекст по слову (без разницы на позицию).
<br><br>

2. Continuous Bag of Words (CBOW). 
    
    Предсказывать слово из BOW-контекста.

Два (средне эффективных) метода обучения:

1. Иерархический софтмакс.
2. Negative sampling. 

__Naive softmax__

### Разбор 

$$ J(\theta) = -\frac{1}{T}\sum_{t=1}^T \sum_{-m \le j \le m, j \ne 0} \log p(w_{t+j} \:|\: w_t)$$

$$ p(o \:|\: c) = \frac{\exp(u_o^Tv_c)}{\sum_{w=1}^V \exp(u_w^Tv_c)} $$

, где $ \theta = \begin{bmatrix}
           v_{aardvark} \\
           v_{a} \\
           \vdots \\
           v_{zebra} \\
           u_{aardvark} \\
           u_{a} \\
           \vdots \\
           u_{zebra}
         \end{bmatrix} \in \mathbb{R}^{2dV} $, $c$ - center indices, $o$ - outside indices.

##### Решение

Чтобы заработал градиентный спуск, надо взять две производные по $v$ и по $u$, и спускаться с шагом обучения. И раз $J(\theta)$ у нас задан, приступим:

$$ \frac{\partial}{\partial v_c} \log \frac{\exp(u_o^Tv_c)}{\sum_{w=1}^V \exp(u_w^Tv_c)} =$$

$$ = \frac{\partial}{\partial v_c} \log \exp(u_o^Tv_c) - \frac{\partial}{\partial v_c} \log \sum_{w=1}^V \exp(u_w^Tv_c) =$$

$$ = \frac{\partial}{\partial v_c} u_o^Tv_c - \Big( \frac{1}{\sum_{w=1}^V \exp(u_w^Tv_c) } \Big) \frac{\partial}{\partial v_c} \sum_{x=1}^V \exp (u_x^Tv_c) =$$

$$ = u_o - \Big( \frac{1}{\sum_{w=1}^V \exp(u_w^Tv_c) } \Big) \sum_{x=1}^V \frac{\partial}{\partial v_c} \exp (u_x^Tv_c) =$$

$$ = u_o - \Big( \frac{1}{\sum_{w=1}^V \exp(u_w^Tv_c) } \Big) \sum_{x=1}^V \exp(u_x^Tv_c) \frac{\partial}{\partial v_c} u_x^Tv_c = $$

$$ = u_o - \Big( \frac{1}{\sum_{w=1}^V \exp(u_w^Tv_c) } \Big) \sum_{x=1}^V \exp(u_x^Tv_c) \: u_x = $$

$$ = u_o - \sum_{x=1}^V \frac{\exp(u_x^Tv_c)}{\sum_{w=1}^V \exp(u_w^Tv_c) } =$$

$$ = u_o - \sum_{x=1}^V p(x \:|\: c) \: u_x $$

Получили разницу между observed вектором и векторами всех остальных слов, взвешенными правдоподобностью возникновения в контексте.

Производная по $u$ - домашка :(

## Разбор статьи


* [Arora __"A Simple but Tough-to-Beat Baseline for Sentence Embeddings"__](https://openreview.net/pdf?id=SyK00v5xx)

__Старый подход в BOW:__ 

`v("one two three") = 1/3\*(v("one") + v("two") + v("three"))`

__Новый подход:__

* __Шаг 1.__ Для каждого предложения $s$ в тексте $S$:

$$ v_s \leftarrow \frac{1}{|s|}\sum_{w \in s} \frac{a}{a+p(w)}v_w $$

Здесь $a$ выполняет роль регуляризации. В целом у шага 1 интуиция во взвешивании слов по их частоте.

* __Шаг 2.__ Вычисли первую PCA-компоненту $u$ для каждого $ \{ v_s \::\: s \in S \} $. Далее для каждого предложения $s$ в тексте $S$:

$$ v_s \leftarrow v_s - uu^Tv_s $$